In [ ]:
#importing the necessary libraries
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0   # or ResNet50, VGG16 etc.
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
#loading the dataset for training
train_ds = tf.keras.utils.image_dataset_from_directory(
    "DDI/",
    labels="inferred",        
    label_mode="int",        
    image_size=(224, 224),   
    batch_size=32,          
    shuffle=True,           
    seed=42                   
)

In [ ]:
# Training and validation split
val_ds = tf.keras.utils.image_dataset_from_directory(
    "DDI/",
    labels="inferred",
    label_mode="int",
    image_size=(224, 224),
    batch_size=32,
    validation_split=0.2,
    subset="validation",
    seed=42
)

train_ds = tf.keras.utils.image_dataset_from_directory(
     "DDI/",
    labels="inferred",
    label_mode="int",
    image_size=(224, 224),
    batch_size=32,
    validation_split=0.2,
    subset="training",
    seed=42
)


In [ ]:
#Autotune
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# data augmentation
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [ ]:
# Setting up the CNN
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)), 
    data_augmentation,                                   

    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  
])

In [ ]:
#compiling the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# 
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
) 